## Set file categories from title

##### functions designed for "df_grouped_NAID_sorted_title.parquet"

In [415]:
import re
import pandas as pd

In [416]:
df = pd.read_parquet('df_grouped_NAID_sorted_title.parquet')

#### Determine which files have common application title format by adding temporary 'file_type' column

In [417]:
# file type groupings - column 'file_type'
application_prefix = 'revolutionary war pension and bounty land warrant application file' 
# family_record = 'family record' 
# microfilm_target_sheet = 'microfilm target sheet'
# other = 'other'

In [418]:
df['file_type'] = ''

In [419]:
df.loc[(df['file_type'] == '') & df['title'].str.replace('-', ' ').str.lower().str.contains(application_prefix, case=False, na=False), 'file_type'] = application_prefix

In [420]:
# Count how many rows have file_type = application_prefix
# count = (df['file_type'] == application_prefix).sum()
# print(f"Number of rows with file_type = application_prefix: {count}")
# print(f"Percentage: {count / len(df) * 100:.2f}%")

In [421]:
# Show a few rows where file_type == application_prefix
# application_rows = df[df['file_type'] == application_prefix]
# application_rows.head()

#### Parse title field to get category by adding temporary 'title_modified' column

In [422]:
df['title_modified'] = ''

# Only create title_modified for rows where file_type is application_prefix
mask = df['file_type'] == application_prefix

df.loc[mask, 'title_modified'] = (df.loc[mask, 'title']
    .str.replace('-', ' ')
    .str.lower()
    .str.replace(application_prefix + 's', application_prefix, case=False, regex=False)
    .str.replace(application_prefix, '', regex=True) # remove application_prefix
    .str.replace('illustrated family record (fraktur) found in', '', regex=False)
    .str.replace(' +', ' ', regex=True) # replace multiple consecutive spaces with a single space
    .str.strip() # remove leading and trailing whitespace
)

In [423]:
# Sort rows with file_type == application_prefix by first 20 chars of title_modified
mask = df['file_type'] == application_prefix 

# Create a temporary column with first 20 characters for sorting
df.loc[mask, 'sort_key'] = df.loc[mask, 'title_modified'].str[:20]

# Sort the entire dataframe, but only the application rows will be sorted by this key
df = df.sort_values(by=['file_type', 'sort_key'], ascending=[False, True])

# Drop the temporary sort_key column
df = df.drop(columns=['sort_key'])

In [424]:
# df.columns


In [425]:
# Display more sorted application rows
# pd.set_option('display.max_colwidth', None)

# application_rows = df[(df['file_type'] == application_prefix) & (df['title_modified'].notna()) & (df['title_modified'] != '')]

# see beginning
# application_rows[['title_modified', 'naraURL']].head(50)

# see end
# application_rows[['title_modified', 'naraURL']].tail(50)

# see middle
# total_rows = len(application_rows)
# middle_start = total_rows // 2 - 34000 
# application_rows[['title_modified', 'naraURL']].iloc[middle_start:middle_start + 50]

### Add file_cat column to set category based on title (e.g. soldier, widow, etc.)

In [426]:
df['file_cat'] = ''

In [427]:
# if file_type is not application_prefix, set file_cat to non_application
if (df['file_type'] != application_prefix).any():
    df.loc[df['file_type'] != application_prefix, 'file_cat'] = 'non_application'

In [428]:
# Only process rows where file_type is application_prefix and title_modified is not empty
mask = df['file_type'] == application_prefix

In [429]:
# success means title_modified starts with one of these and then is directly followed by optional space and at least 3 numbers
category_dict = {
    "soldier": ['s', 's.'], # soldier (survived)
    'rejected': ['r', 'r.'], # rejected
    "widow": ['w', 'w.', 'wid', 'w;', 'wid ctf.', 'w. file', 'w .', 'w,'], # widow
    "bounty land warrant": ['wt.', 'wt', 'b.l.wt.', 'b . l. wt.', 'b .l. wt.', 'b l reg.', 'b l wt', 'b l. wt.', 'b. l.', 'b l.wt.', 'b l wt'], # bounty land warrant
    "N A Acc": ['acc. no.', 'n.a.acc. no.', "n a acc no", "n a acc", "n a acc no.", "acc no"], # National Archives Accession [number]
}

In [430]:
# Iterate through category_dict and set file_cat based on pattern matching
for category_key, values in category_dict.items():
    for value in values:
        # string starts with dictionary value, then optional space, then next char needs to be a digit, and the following 2 chars can be either digit, comma, period or space
        escaped_value = re.escape(value)
        pattern = f'^{escaped_value}\\s*\\d[\\d,\\.\\s]{{2}}'
        
        # Check if title_modified matches the pattern
        value_mask = df['title_modified'].str.match(pattern, case=False, na=False)
        
        # Set file_cat where pattern matches and mask is True
        df.loc[mask & value_mask & (df['file_cat'] == ''), 'file_cat'] = category_key

In [431]:
# # Another pass to catch random punctuation for bounty land warrant cases - basically any combination of 'b l' or 'b l wt' in correct order with various punctuation
# # Check for 'b.l.wt' pattern and set file_cat to 'bounty land warrant'
mask = (df['file_type'] == application_prefix) & (df['file_cat'] == '')

# blwt_pattern = r"^b(?:[ .,]|[.,] ?| ?[.,])?l(?:[ .,]|[.,] ?| ?[.,])?wt[.,]?"
blwt_pattern = r"^b(?:[ .,]|[.,] ?| ?[.,])?l(?:(?:[ .,]|[.,] ?| ?[.,])?wt[.,]?)?"
blwt_pattern_mask = df['title_modified'].str.match(blwt_pattern, case=False, na=False)

# Set file_cat to 'bounty land warrant' where pattern matches and file_cat is empty
df.loc[mask & blwt_pattern_mask, 'file_cat'] = 'bounty land warrant'



# # Another pass to catch random punctuation for "N A Acc" cases - basically any combination of 'n a acc' or 'acc no' in correct order with various punctuation
pattern_na_acc = r"^n(?:[ .,]|[.,] ?| ?[.,])?a(?:[ .,]|[.,] ?| ?[.,])?acc[.,]?"
pattern_acc_no = r"^acc(?:[ .,]|[.,] ?| ?[.,])?no[.,]?"
naac_pattern_mask = df['title_modified'].str.match(pattern_na_acc, case=False, na=False) | df['title_modified'].str.match(pattern_acc_no, case=False, na=False)

df.loc[mask & naac_pattern_mask, 'file_cat'] = "N A Acc"

In [ ]:
# success means that any of these phrases are included
category_dict_special_cases = {
    'old war': ['old war', 'not rev. war', 'not. rev. war', 'not rev war', 'indian war', 'ind. war', 'o.w.', 'o. w.'],
    'nara archival administrative sheets': ['nara archival administrative sheets'],
    'microfilm': ['microfilm']
}

In [433]:
# Iterate through category_dict_special_cases and append matching categories - allow to append a second category (mostly for old war cases even if one is alredy set by the standard dictionary)
for category_key, phrases in category_dict_special_cases.items():
    for phrase in phrases:
        phrase_mask = df['title_modified'].str.contains(phrase, case=False, na=False, regex=False)
        matching_rows = mask & phrase_mask
        
        # For each matching row, check if file_cat already has a value
        for idx in df[matching_rows].index:
            current_value = df.loc[idx, 'file_cat']
            if current_value == '':
                # If empty, just set it
                df.loc[idx, 'file_cat'] = category_key
            else:
                # If not empty, split on '||', add new category if not present, remove duplicates, and re-concatenate
                categories = current_value.split('||')
                if category_key not in categories:
                    categories.append(category_key)
                    df.loc[idx, 'file_cat'] = '||'.join(sorted(categories))

Unkown options - set `file_cat` to `'unknown'` if fits one of these cases
1. if `title_modified` is empty
2. if `title_modified` starts with `'[blank]'` or `'[illegible]'`
<!-- 3. if `title_modified` starts with at least 4 characters that are a combination of numbers, spaces and punctuation -->

In [434]:
# unknown options - set file_cat to 'unknown' if fits one of these cases
# Only set for application rows where file_cat is not already set
mask_unknown = (df['file_type'] == application_prefix) & (df['file_cat'] == '')

# Case 1: if title_modified is empty
empty_mask = mask_unknown & ((df['title_modified'].isna()) | (df['title_modified'].str.strip() == ''))
df.loc[empty_mask, 'file_cat'] = 'unknown'

# Case 2: if title_modified starts with '[blank]' or '[illegible]' or 'for'
blank_illegible_mask = mask_unknown & (
    df['title_modified'].str.startswith('[blank]', na=False) | 
    df['title_modified'].str.startswith('[illegible]', na=False) |
    df['title_modified'].str.startswith('for', na=False)
)
df.loc[blank_illegible_mask, 'file_cat'] = 'unknown'

In [435]:
# check for duplicates (ie. 
# wt. 7554 100 wt. 7555 100, for john neilson, [blank]
#)
#

### Check results

In [ ]:
# Check percentage of rows with file_cat defined
total_rows = len(df)
rows_with_file_cat = ((df['file_cat'].notna()) & (df['file_cat'] != '')).sum()
percentage = (rows_with_file_cat / total_rows) * 100

print(f"Total rows: {total_rows}")
print(f"Rows with file_cat defined: {rows_with_file_cat}")
print(f"Percentage: {percentage:.2f}%")

# Check file_cat distribution
print("File_cat value counts:")
print(df['file_cat'].value_counts(dropna=False))
print(f"\nTotal rows: {len(df)}")
print(f"Rows with file_cat defined: {((df['file_cat'].notna()) & (df['file_cat'] != '')).sum()}")
print(f"Percentage: {((df['file_cat'].notna()) & (df['file_cat'] != '')).sum() / len(df) * 100:.2f}%")

# Check percentage for application rows only
mask = df['file_type'] == application_prefix
app_rows = df[mask]
rows_with_file_cat = ((app_rows['file_cat'].notna()) & (app_rows['file_cat'] != '')).sum()
percentage = (rows_with_file_cat / len(app_rows)) * 100

print(f"Application rows: {len(app_rows)}")
print(f"Application rows with file_cat defined: {rows_with_file_cat}")
print(f"Percentage: {percentage:.2f}%")

# percentage of each category

Total rows: 78926
Rows with file_cat defined: 78596
Percentage: 99.58%
File_cat value counts:
file_cat
soldier                                35960
widow                                  25517
rejected                               11034
bounty land warrant                     3238
unknown                                 2152
                                         330
nara archival administrative sheets      303
non_application                          302
old war                                   67
bounty land warrant||old war              12
N A Acc                                   11
Name: count, dtype: int64

Total rows: 78926
Rows with file_cat defined: 78596
Percentage: 99.58%
Application rows: 78624
Application rows with file_cat defined: 78294
Percentage: 99.58%


In [442]:
# percentage of each category
category_counts = df['file_cat'].value_counts(dropna=False)
category_percentages = (category_counts / len(df) * 100).round(2)

print("Percentage of each category:")
for category, percentage in category_percentages.items():
    print(f"{category}: {percentage}%")

Percentage of each category:
soldier: 45.56%
widow: 32.33%
rejected: 13.98%
bounty land warrant: 4.1%
unknown: 2.73%
: 0.42%
nara archival administrative sheets: 0.38%
non_application: 0.38%
old war: 0.08%
bounty land warrant||old war: 0.02%
N A Acc: 0.01%


In [438]:
# Show application rows with blank file_cat
pd.set_option('display.max_colwidth', None)

mask = (df['file_type'] == application_prefix) & (df['file_cat'] == '')
blank_app = df[mask]
print(f"Application rows with blank file_cat: {len(blank_app)}")
blank_app[['title_modified',  'file_cat', 'naraURL']].tail(50)

Application rows with blank file_cat: 330


,title_modified,file_cat,naraURL
33197,"s., for william h price, north carolina",,https://catalog.archives.gov/id/196440777
37553,"s., for william snodgrass, virginia",,https://catalog.archives.gov/id/196685315
38748,"s., for william stevens, massachusetts",,https://catalog.archives.gov/id/196719984
72321,"s., james hall, n.h.",,https://catalog.archives.gov/id/54764290
56193,"s., john combs, continental mass. n.h.",,https://catalog.archives.gov/id/54274956
679,"s., john tucker, n.c.",,https://catalog.archives.gov/id/111424046
1662,"s., john van houten, continental new york",,https://catalog.archives.gov/id/111454367
7834,"s., john wilson, n.c.",,https://catalog.archives.gov/id/111740546
4893,"s., john yount, penn.",,https://catalog.archives.gov/id/111649319
59864,"s., josiah davis, conn.",,https://catalog.archives.gov/id/54386126


In [439]:
# save df to parquet
df.to_parquet('df_grouped_NAID_sorted_title_with_file_cat.parquet', engine='pyarrow')

In [441]:
# join df with nara_pension_file_pages.parquet on NAID - keep columns of nara_pension_file_pages and only add file_cat column

# Read the nara_pension_file_pages.parquet file
nara_df = pd.read_parquet('nara_pension_file_pages.parquet')

# Merge the two dataframes on NAID
merged_df = pd.merge(nara_df, df[['NAID', 'file_cat']], on='NAID', how='left')

# Save the merged dataframe to a new parquet file
merged_df.to_parquet('nara_pension_file_pages_with_file_cat.parquet', engine='pyarrow')